In [2]:
from typing import List, Optional, Literal
from pydantic import BaseModel, Field
from dslmodel import DSLModel
from enum import Enum


class LaTeXComponentType(str, Enum):
    SECTION = "section"
    SUBSECTION = "subsection"
    EQUATION = "equation"
    FIGURE = "figure"
    TABLE = "table"
    REFERENCE = "reference"


class LaTeXComponent(DSLModel):
    """Base class for all LaTeX components."""
    type_: LaTeXComponentType = Field(..., alias="type", description="Type of the LaTeX component.")
    content: Optional[str] = Field(None, description="Content of the LaTeX component.")
    label: Optional[str] = Field(None, description="Label for referencing.")
    
    def to_latex(self) -> str:
        """Convert component to LaTeX format."""
        return self.content or ""


class Section(LaTeXComponent):
    """Represents a section in LaTeX."""
    type_: Literal[LaTeXComponentType.SECTION] = Field(LaTeXComponentType.SECTION, alias="type", description="Type of the LaTeX component.")
    title: str = Field(..., description="Title of the section.")
    content: List[LaTeXComponent] = Field(default_factory=list, description="Content of the section, which may include subsections, equations, etc.")

    def to_latex(self) -> str:
        """Render the section as LaTeX code."""
        latex_content = f"\\section{{{self.title}}}\n"
        for item in self.content:
            latex_content += item.to_latex()
        return latex_content


class Subsection(LaTeXComponent):
    """Represents a subsection in LaTeX."""
    type_: Literal[LaTeXComponentType.SUBSECTION] = Field(LaTeXComponentType.SUBSECTION, alias="type", description="Type of the LaTeX component.")
    title: str = Field(..., description="Title of the subsection.")
    content: List[LaTeXComponent] = Field(default_factory=list, description="Content of the subsection.")

    def to_latex(self) -> str:
        """Render the subsection as LaTeX code."""
        latex_content = f"\\subsection{{{self.title}}}\n"
        for item in self.content:
            latex_content += item.to_latex()
        return latex_content


class Equation(LaTeXComponent):
    """Represents an equation in LaTeX."""
    type_: Literal[LaTeXComponentType.EQUATION] = Field(LaTeXComponentType.EQUATION, alias="type", description="Type of the LaTeX component.")
    equation: str = Field(..., description="Equation content in LaTeX format.")
    label: Optional[str] = Field(None, description="Label for referencing the equation.")

    def to_latex(self) -> str:
        """Render the equation as LaTeX code."""
        if self.label:
            return f"\\begin{{equation}}\\label{{{self.label}}}\n{self.equation}\n\\end{{equation}}\n"
        else:
            return f"\\begin{{equation}}\n{self.equation}\n\\end{{equation}}\n"


class Figure(LaTeXComponent):
    """Represents a figure in LaTeX."""
    type_: Literal[LaTeXComponentType.FIGURE] = Field(LaTeXComponentType.FIGURE, alias="type", description="Type of the LaTeX component.")
    image_path: str = Field(..., description="Path to the image file.")
    caption: str = Field(..., description="Caption for the figure.")
    label: Optional[str] = Field(None, description="Label for referencing the figure.")

    def to_latex(self) -> str:
        """Render the figure as LaTeX code."""
        latex_content = f"\\begin{{figure}}[h!]\n\\centering\n\\includegraphics[width=0.8\\textwidth]{{{self.image_path}}}\n\\caption{{{self.caption}}}\n"
        if self.label:
            latex_content += f"\\label{{{self.label}}}\n"
        latex_content += "\\end{figure}\n"
        return latex_content


class Table(LaTeXComponent):
    """Represents a table in LaTeX."""
    type_: Literal[LaTeXComponentType.TABLE] = Field(LaTeXComponentType.TABLE, alias="type", description="Type of the LaTeX component.")
    rows: List[List[str]] = Field(..., description="Rows of the table.")
    caption: Optional[str] = Field(None, description="Caption for the table.")
    label: Optional[str] = Field(None, description="Label for referencing the table.")

    def to_latex(self) -> str:
        """Render the table as LaTeX code."""
        latex_content = "\\begin{table}[h!]\n\\centering\n\\begin{tabular}{| " + " | ".join("c" for _ in self.rows[0]) + " |}\n\\hline\n"
        for row in self.rows:
            latex_content += " & ".join(row) + " \\\\\n\\hline\n"
        latex_content += "\\end{tabular}\n"
        if self.caption:
            latex_content += f"\\caption{{{self.caption}}}\n"
        if self.label:
            latex_content += f"\\label{{{self.label}}}\n"
        latex_content += "\\end{table}\n"
        return latex_content


class Reference(LaTeXComponent):
    """Represents a reference in LaTeX."""
    type_: Literal[LaTeXComponentType.REFERENCE] = Field(LaTeXComponentType.REFERENCE, alias="type", description="Type of the LaTeX component.")
    label: str = Field(..., description="Label for the reference.")
    description: Optional[str] = Field(None, description="Description or citation text.")

    def to_latex(self) -> str:
        """Render the reference as LaTeX code."""
        return f"\\ref{{{self.label}}} {self.description or ''}"


class LaTeXDocument(DSLModel):
    """Represents a complete LaTeX document."""
    title: str = Field(..., description="Title of the document.")
    author: Optional[str] = Field(None, description="Author of the document.")
    sections: List[Section] = Field(default_factory=list, description="List of sections in the document.")

    def render(self) -> str:
        """Render the document as LaTeX code."""
        latex_output = f"\\title{{{self.title}}}\n\\author{{{self.author}}}\n\\begin{{document}}\n\\maketitle\n"
        for section in self.sections:
            latex_output += section.to_latex()
        latex_output += "\\end{document}"
        return latex_output


In [3]:
# Create components
equation = Equation(equation="E = mc^2", label="eq:energy")
figure = Figure(image_path="images/diagram.png", caption="A sample diagram.", label="fig:diagram")
table = Table(rows=[["Header 1", "Header 2"], ["Row 1 Col 1", "Row 1 Col 2"]], caption="Sample Table", label="tab:sample")

# Define sections and subsections
subsec = Subsection(title="Key Concepts", content=[equation, figure, table])
section = Section(title="Introduction", content=[subsec])

# Create document
doc = LaTeXDocument(
    title="Sample Document",
    author="John Doe",
    sections=[section]
)

# Render LaTeX code
print(doc.render())


In [5]:
LaTeXComponent.__repr_name__